# Data Preparation - Instacart Market Basket Analysis

## Overview

This notebook focuses on cleaning and preparing the Instacart datasets for analysis. Based on the findings from our initial data exploration, we will address data quality issues and ensure our datasets are ready for comprehensive analysis.

## Objectives

**1. Data Quality Assessment**
- Handle missing values across all datasets
- Remove or flag duplicate records
- Validate data consistency and logical constraints

**2. Data Cleaning**
- Clean inconsistent data formats
- Standardize data types where necessary
- Address any anomalies identified during exploration

**3. Data Integration**
- Merge datasets where appropriate for analysis
- Create derived variables if needed
- Ensure referential integrity between tables

**4. Data Validation**
- Verify cleaned data meets quality standards
- Perform final consistency checks
- Document all transformations applied

## Expected Outcome

Clean, validated datasets ready for in-depth analysis of customer shopping patterns, product preferences, and ordering behaviors.

---

*Note: All cleaning steps will be documented with clear explanations and rationale for reproducibility.*

In [55]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [56]:
# Reading the data to variables
df_instacart_orders = pd.read_csv('../data/raw/instacart_orders.csv', sep=';')
df_products = pd.read_csv('../data/raw/products.csv', sep=';')
df_order_products = pd.read_csv('../data/raw/order_products.csv', sep=';')
df_aisles = pd.read_csv('../data/raw/aisles.csv', sep=';')
df_departments = pd.read_csv('../data/raw/departments.csv', sep=';')

### DataFrame instacart_orders

In [57]:
#count duplicated orders
dup_num = df_instacart_orders.duplicated().sum()
message = f"df_instacart_orders have {dup_num} duplicated lines"
print(message)

print("===========================================")
#printing duplicated rows sorted by order_id
dup_rows = df_instacart_orders[df_instacart_orders.duplicated(keep=False)]
print(dup_rows.sort_values(by='order_id'))

df_instacart_orders have 15 duplicated lines
        order_id  user_id  order_number  order_dow  order_hour_of_day  \
354993    391768    57671            19          3                  2   
371905    391768    57671            19          3                  2   
119251    408114    68324             4          3                  2   
321100    408114    68324             4          3                  2   
394347    467134    63189            21          3                  2   
250626    467134    63189            21          3                  2   
145574    794638    50898            24          3                  2   
99462     794638    50898            24          3                  2   
242618   1021560    53767             3          3                  2   
311713   1021560    53767             3          3                  2   
273805   1112182   202304            84          3                  2   
164581   1112182   202304            84          3                  2   
314427

#### DataFrame instacart_orders Analysis
- 15 duplicated orders
- All happened on Wednesday (order_dow == 3) at 2am (order_hour_of_day == 2)

#### Identified Issues
1. Lines are matching several fields (order_id, user_id, order_number)
2. The same user_id cannot place the same order_id multiple times
3. order_id must be a unique key value (not repeated)

#### Conclusion
We can conclude this probably happened due to a server failure or an error during backup, so we must remove the duplicated lines. Despite representing a residual number of lines, we must keep data integrity in mind.

In [58]:
#removing duplicates and reset index
df_instacart_orders_clean = df_instacart_orders.drop_duplicates().reset_index(drop = True)

#count duplicated orders
dup_num = df_instacart_orders_clean.duplicated().sum()
message = f"df_instacart_orders_cleared have {dup_num} duplicated lines"
print(message)

#checking duplicated order_id
dup_order_id = df_instacart_orders_clean['order_id'].duplicated().sum()
message = f"df_instacart_orders_cleared have {dup_order_id} duplicated order_id"
print(message)

df_instacart_orders_cleared have 0 duplicated lines
df_instacart_orders_cleared have 0 duplicated order_id


### DataFrame products


In [59]:
# change product_name to lowercase
df_products['product_name'] = df_products['product_name'].str.lower()

# verify duplicated lines
products_dup_lines = df_products.duplicated().sum()
message = f"products dataframe have {products_dup_lines} duplicated lines"
print(message)

# verify duplicated values in column produt_id
product_id_dup = df_products['product_id'].duplicated().sum()
message = f"product_id column have {product_id_dup} duplicated values"
print(message)

# verify duplicated in product_names
product_name_dups = df_products['product_name'].duplicated().sum()
message = f"product_name column have {product_name_dups} duplicated values"
print(message)

print(df_products[df_products['product_name'].isna()])

products dataframe have 0 duplicated lines
product_id column have 0 duplicated values
product_name column have 1361 duplicated values
       product_id product_name  aisle_id  department_id
37             38          NaN       100             21
71             72          NaN       100             21
109           110          NaN       100             21
296           297          NaN       100             21
416           417          NaN       100             21
...           ...          ...       ...            ...
49552       49553          NaN       100             21
49574       49575          NaN       100             21
49640       49641          NaN       100             21
49663       49664          NaN       100             21
49668       49669          NaN       100             21

[1258 rows x 4 columns]


#### DataFrame products Analysis
- 0 duplicated full lines (good data quality)
- 0 duplicated product_id, good as each product has unique identifier
- 1 361 duplicated values in column "product_name" due to NaN values

#### Identified Issues
**Missing Product Names**:
   - These all belong to aisle_id = 100 and department_id = 21
   - This suggests a systematic data collection issue for products in this specific category
   - Likely represents "missing" or "unknown" products in the system



#### Conclusion
We can conclude this missing values are related to a especific Aisle (100) /Department(21) so is better to investigate what they represent in the businness. The collection of this especific data must be reviewd. For further analisys the NaN values should be replaced by "Unknown Product" or similar placeholder.

In [60]:
# replacing the NaN values
df_products_clean = df_products[df_products['product_name'].duplicated() == True].fillna('unknown product')

#confirm change
print(df_products_clean[df_products_clean['product_name'] == 'unknown product'])

       product_id     product_name  aisle_id  department_id
71             72  unknown product       100             21
109           110  unknown product       100             21
296           297  unknown product       100             21
416           417  unknown product       100             21
436           437  unknown product       100             21
...           ...              ...       ...            ...
49552       49553  unknown product       100             21
49574       49575  unknown product       100             21
49640       49641  unknown product       100             21
49663       49664  unknown product       100             21
49668       49669  unknown product       100             21

[1257 rows x 4 columns]


### DataFrame departments


In [61]:
# verify duplicated lines
departments_dup_lines = df_departments.duplicated().sum()
message = f"departments dataframe have {products_dup_lines} duplicated lines"
print(message)

# verify duplicated id
department_id_dup = df_departments['department_id'].duplicated().sum()
message = f"departments dataframe have {department_id_dup} duplicated ids"
print(message)

departments dataframe have 0 duplicated lines
departments dataframe have 0 duplicated ids


#### DataFrame department Analysis
- 0 duplicated full lines (good data quality)
- 0 duplicated product_id, good as each department has unique identifier

#### Identified Issues
**No issues identified**

#### Conclusion
We can conclude the department list have quality data, but we can also see the problem discovred before with department 21 is the last line.

### DataFrame aisles


In [62]:
# verify duplicated lines
aisles_dup_lines = df_aisles.duplicated().sum()
message = f"aisles dataframe have {aisles_dup_lines} duplicated lines"
print(message)

# verify duplicated id
aisles_id_dup = df_aisles['aisle_id'].duplicated().sum()
message = f"aisles dataframe have {aisles_id_dup} duplicated ids"
print(message)

aisles dataframe have 0 duplicated lines
aisles dataframe have 0 duplicated ids


#### DataFrame aisles Analysis
- 0 duplicated full lines (good data quality)
- 0 duplicated aisle_id, good as each aisle has unique identifier

#### Identified Issues
**No issues identified**

#### Conclusion
We can conclude the aisles list have quality data.

### DataFrame order_products


In [63]:
# verify duplicated lines
order_prod_dup_lines = df_order_products.duplicated().sum()
message = f"order_products dataframe have {order_prod_dup_lines} duplicated lines"
print(message)

# min and max values for 'add_to_cart_order'
min_orders = df_order_products['add_to_cart_order'].min()
max_orders = df_order_products['add_to_cart_order'].max()
message = f"The smallest order have {min_orders} product(s) and the largest have {max_orders} prduct(s)"
print(message)

# saving lines with NaN in 'add_to_cart_order'
missing_values = df_order_products[df_order_products['add_to_cart_order'].isna() == True]
#print(missing_values.sort_values(by='order_id'))

# using one of the order_id as example to check max add_to_cart_order
filtred_df = df_order_products[df_order_products['order_id'] == 61355].sort_values(by='add_to_cart_order')
max_add_to_cart = filtred_df['add_to_cart_order'].max()
print(f'orders with NaN values have a max of {max_add_to_cart} lines')

order_products dataframe have 0 duplicated lines
The smallest order have 1.0 product(s) and the largest have 64.0 prduct(s)
orders with NaN values have a max of 64.0 lines


In [64]:
# group by order_id the lines with NaN values
nan_grouped = missing_values.groupby('order_id').agg({
    'product_id': 'count',  # Count how many products per order have NaN
}).rename(columns={'product_id': 'nan_count'})

print("Orders with NaN values grouped by order_id:")
print(nan_grouped.head(10))

Orders with NaN values grouped by order_id:
          nan_count
order_id           
9310              1
61355            63
102236           31
129627            5
165801            6
171934           40
180546            2
264710           27
293169           13
388234            6


In [70]:
# count 'prouct_id' number per order
df_order_products.groupby('order_id')['product_id'].count()

order_id
4          13
9          15
11          5
19          3
20          8
           ..
3421034    17
3421053     9
3421071     5
3421077     4
3421079     1
Name: product_id, Length: 450046, dtype: int64

In [71]:
# count 'prouct_id' number per order
grouped = df_order_products.groupby('order_id')['product_id'].count()
print(grouped.min())

1


#### DataFrame order_products Analysis
- 0 duplicated full lines, meaning no errors when saving the data
- orders vary bettween 1 and 64 products
- orders with more then 64 products have NaN values in column 'add_to_cart_order'


#### Identified Issues
- feeding issue for order_id with more than 64 products, resulting in NaN values in add_to_cart_order column.

#### Conclusion
Dataframe have no duplicated lines which pass a firsts impression that all is good, but analyzing deeper we could realize a issue with orders with more than 64 products.This limit suggests a technical constraint in the cart system.

In [69]:
# changing 'reorder' to type bool
df_order_products['reordered'] = df_order_products['reordered'].astype('bool')

# replacing NaN values in add_to_cart_order to 999
df_order_products_clean = df_order_products[df_order_products['add_to_cart_order'].isna() == True].fillna('999')

# changing 'add_to_cart_order' to type int
df_order_products_clean['add_to_cart_order'] = df_order_products_clean['add_to_cart_order'].astype('int64')


In [72]:
# SAVING CLEANED DATASETS FOR ANALYSIS
df_instacart_orders_clean.to_csv('../data/data_clean/instacart_orders_clean.csv', index=False)
df_products_clean.to_csv('../data/data_clean/products_clean.csv', index=False)
df_order_products_clean.to_csv('../data/data_clean/order_products_clean.csv', index=False)